### Procédure de test et fonctionnement de `partie_electroaimant.py` (Objectif 1)

**Contexte :** Ce fichier modélise la trajectoire d'une particule chargée se déplaçant initialement selon +y et entrant dans un champ magnétique uniforme `Bz` orienté selon +z.

**Paramètres de base pour les tests (sauf indication contraire) :**
- `masse_charge` = (1.0, 1.0) (masse=1.0 u, charge=1.0 e)
- `v_initiale` = 1e6 m/s
- `Bz_test` = 0.1 T
- `x_test` = 0.01 m
- `mq_ref` ≈ 1.0364e-8 kg/C (pour (1.0 u, 1.0 e))

**1. Tests de la Classe `particule`**

| ID   | Fonctionnalité                     | Procédure                                                                                                   | Données de test                                     | Résultat attendu                                                                                                                              |
| :--- | :--------------------------------- | :---------------------------------------------------------------------------------------------------------- | :-------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------- |
| 1 | Initialisation valide              | `p = particule(masse_charge=(1.0, 1.0), v_initiale=1e6)`                                                    | masse=1.0u, charge=1.0e, v0=1e6 m/s                | Objet `p` créé. `p.m=1.0`, `p.charge_affichage=1.0`, `p.vo=1e6`. `p.mq` ≈ `mq_ref`.                                                            |
| 2 | Initialisation charge négative     | `p = particule(masse_charge=(1.0, -1.0), v_initiale=1e6)`                                                   | masse=1.0u, charge=-1.0e, v0=1e6 m/s               | Objet `p` créé. `p.charge_affichage=-1.0`. `p.mq` calculé avec `abs(charge_e)` (donc `p.mq` ≈ `mq_ref`).                                    |
| 3 | `equation_trajectoire` valide      | `p = particule((1,1), 1e6)`. Calculer `p.equation_trajectoire(x=0.01, Bz=0.1)`                               | x=`x_test`, Bz=`Bz_test`                             | Retourne une valeur `y` float (approx. 0.00484 m).                                                                                             |
| 4 | `equation_trajectoire` (arg `arccos` > 1) | `p = particule((1,1), 1e6)`. Calculer `p.equation_trajectoire(x=0.5, Bz=0.1)` (x trop grand pour R cyclotron) | x=0.5m, Bz=0.1T                                     | Retourne `NaN` (dû à `np.arccos` d'une valeur > 1, géré par `np.errstate(invalid='ignore')`).                                                   |
| 5 | `equation_trajectoire` (Bz=0)      | `p = particule((1,1), 1e6)`. Calculer `p.equation_trajectoire(x=0.01, Bz=0)`                                 | x=`x_test`, Bz=0T                                   | Lève `RuntimeWarning: divide by zero encountered in true_divide` (dans `prefix`) et retourne `NaN`.                                         |
| 6 | `equation_trajectoire` (x=0)       | `p = particule((1,1), 1e6)`. Calculer `p.equation_trajectoire(x=0, Bz=0.1)`                                 | x=0m, Bz=`Bz_test`                                   | Retourne 0.0 (car `np.arccos(1)` = 0, `np.sin(0)` = 0).                                                                                       |
| 7 | `trajectoire` valide               | `p = particule((1,1), 1e6)`. Calculer `x_vals, y_vals = p.trajectoire(Bz=0.1, x_min=0, x_max=0.02, n_points=100)` | Bz=`Bz_test`, x_min=0, x_max=0.02, n=100            | Retourne deux `np.ndarray` `x_vals` et `y_vals`, chacun de taille 100. `x_vals` va de 0 à 0.02.                                                  |
| 8 | `trajectoire` (x_min > x_max)      | `p = particule((1,1), 1e6)`. Calculer `x_vals, y_vals = p.trajectoire(Bz=0.1, x_min=0.02, x_max=0, n_points=100)` | Bz=`Bz_test`, x_min=0.02, x_max=0, n=100           | Retourne deux `np.ndarray` `x_vals` (0.02 à 0) et `y_vals`. Le premier y est celui pour x_min.                                                 |
| 9 | `determiner_champ_magnetique` valide | `p = particule((1,1), 1e6)`. Calculer `p.determiner_champ_magnetique(x_objective=0.01, y_objective=0.005)` | x_obj=0.01m, y_obj=0.005m                           | Retourne une valeur `Bz` float (approx. 0.0964 T).                                                                                             |
| 10| `determiner_champ_magnetique` B0 spécifié | `p = particule((1,1), 1e6)`. Calculer `p.determiner_champ_magnetique(x_objective=0.01, y_objective=0.005, B0=0.1)` | x_obj=0.01m, y_obj=0.005m, B0=0.1T                | Retourne une valeur `Bz` float (approx. 0.0964 T), `fsolve` converge depuis B0.                                                                 |
| 11| `determiner_champ_magnetique` pas de solution réelle | `p = particule((1,1), 1e6)`. Calculer `p.determiner_champ_magnetique(x_objective=0.1, y_objective=0.001)` (y trop petit pour x) | x_obj=0.1m, y_obj=0.001m                         | `fsolve` peut ne pas converger ou retourner une valeur non physique. `RuntimeWarning: invalid value encountered in arccos` peut survenir. Le résultat de `fsolve` peut être la valeur `B0`. |
| 12| `tracer_trajectoire` (simple)      | `fig, ax = plt.subplots(); p = particule((1,1),1e6); p.tracer_trajectoire(ax, Bz=0.1, x_min=0, x_max=0.02, color='r', label='TestP')` | ax valide, Bz=`Bz_test`, x_range, color, label | L'objet `ax` est modifié : contient une ligne rouge avec le label 'TestP'. Pas d'erreur.                                                      |
| 13| `tracer_trajectoire` (x_vals vide) | `fig, ax = plt.subplots(); p = particule((1,1),1e6); p.tracer_trajectoire(ax, Bz=0.1, x_min=0.02, x_max=0.01, n_points=0)` | n_points=0 (résulte en x vide)                    | La fonction retourne sans erreur et `ax` n'est pas modifié (pas de tracé car `len(x)==0`).                                                     |

**2. Tests de la Fonction `tracer_ensemble_trajectoires`**

| ID   | Fonctionnalité                                    | Procédure                                                                                                                                    | Données de test                                                              | Résultat attendu (si `create_plot=False`)                                                                                                                                  |
| :--- | :------------------------------------------------ | :------------------------------------------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1 | `tracer_ensemble_trajectoires` valide             | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires(masses_charges_particules=[(1,1),(2,1)], vitesse_initiale=1e6, Bz=0.1, x_detecteur=0.02, labels_particules=['P1','P2'], ax=ax, create_plot=False)` | Particules (1,1), (2,1), v0=1e6, Bz=`Bz_test`, x_det=0.02, labels | L'objet `ax` est modifié : contient 2 lignes (trajectoires), 1 ligne (détecteur), légende, titre, labels axes. Retourne `None`. Pas d'erreur.                                 |
| 2 | `tracer_ensemble_trajectoires` labels non fournis | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires(masses_charges_particules=[(1,1)], vitesse_initiale=1e6, Bz=0.1, x_detecteur=0.02, ax=ax, create_plot=False)` | Labels non fournis                                                         | `ax` modifié. L'objet `labels` est créé, et `particule_locale.tracer_trajectoire` est appelé avec ce label. Pas d'erreur.                                              |
| 3 | `tracer_ensemble_trajectoires` `create_plot=True` | `tracer_ensemble_trajectoires(masses_charges_particules=[(1,1)], vitesse_initiale=1e6, Bz=0.1, x_detecteur=0.02, create_plot=True)`         | `create_plot=True`                                                           | Une fenêtre Matplotlib s'affiche avec le tracé. La fonction ne retourne rien. Pas d'erreur. (Test manuel pour vérifier l'affichage)                                    |
| 4 | `tracer_ensemble_trajectoires` particules avec y_contact=NaN | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires(masses_charges_particules=[(1,1)], vitesse_initiale=1e6, Bz=0.001, x_detecteur=0.5, ax=ax, create_plot=False)` | Bz très faible, x_detecteur grand (peut mener à `arccos` hors domaine -> NaN) | `ax` modifié. Le détecteur est tracé. Si `all_y_contact` est `[NaN]`, alors `all_y_contact` est redéfini à `[0.07 * x_detecteur]`. Pas d'erreur. |
| 5 | `tracer_ensemble_trajectoires` `ax=None`, `create_plot=False` | `tracer_ensemble_trajectoires(masses_charges_particules=[(1,1)], vitesse_initiale=1e6, Bz=0.1, x_detecteur=0.02, create_plot=False)` | `ax=None`, `create_plot=False`                                               | Une nouvelle figure et un nouvel axe sont créés (`fig, ax = plt.subplots()`). Le tracé est effectué sur cet axe. La figure n'est pas affichée (`plt.show()` n'est pas appelé). |

**3. Test du Bloc `if __name__ == "__main__":` (Exemple d'utilisation)**

| ID   | Fonctionnalité                                  | Procédure                                                                                                  | Données de test                                                                 | Résultat attendu                                                                                                                               |
| :--- | :---------------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------ | :--------------------------------------------------------------------------------------------------------------------------------------------- |
| 1 | Exécution du script principal (si décommenté)   | Décommenter le bloc `if __name__ == "__main__":` et exécuter le fichier.                                    | `rapports_masse_charge = [(1, 1), (2, 1), (3, 1)]`, `v_initiale = 1e7`, `Bz = 1`, `x_detecteur = 1e-4`, `labels_particules=['P1', 'P2', 'P3']` | Le script s'exécute sans erreur. Une fenêtre Matplotlib s'affiche montrant les trajectoires de 3 particules et le détecteur.                    |

*Note :*
*   *Les valeurs numériques exactes pour les "Résultats attendus" peuvent varier légèrement en fonction des constantes physiques précises utilisées par `scipy.constants` et des arrondis.*
*   *Certains cas (comme Bz=0 pour `equation_trajectoire`) mettent en évidence les limites des formules analytiques, qui peuvent ne pas être définies ou numériquement stables dans ces conditions. Le code actuel gère cela avec `np.errstate` pour les `arccos` et les avertissements NumPy pour les divisions par zéro.*
*   *Les tests des fonctions de tracé (marqués M_P12, M_T*) avec `create_plot=False` nécessitent la création manuelle d'une figure et d'un axe Matplotlib (`fig, ax = plt.subplots()`) avant l'appel, sauf si la fonction testée le fait elle-même (comme `tracer_ensemble_trajectoires` lorsque `ax is None`).*